# Exploración del modelo entrenado

In [1]:
import sys
import os 
import torch
import pandas as pd
import torchaudio.transforms as T
import torch.optim as optim
import numpy as np

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

sys.path.append(os.path.abspath(os.path.join('..','iESPnet_SRC_main','utilities')))
from Generator import SeizureDatasetLabelTime, scale_spec, permute_spec, smoothing_label
from Model import iESPnet
from TrainEval import train_model_opt, test_model, train_model, get_thr_output, get_performance_indices
from utilit import make_weights_for_balanced_classes

In [6]:
meta_data_file = '/media/martin/Disco2/Rns_Data/PITT/METADATA/allfiles_metadata.csv'
df_meta = pd.read_csv(meta_data_file)
df_meta.drop(df_meta[df_meta['label'] == 2].index, inplace = True)

In [17]:
# se crea el modelo

N_CLASSES       = 1
learning_rate   = 1e-3
batch_size      = 128
epochs          = 20
num_workers     = 4

hparams = {
        "n_cnn_layers" : 3,
        "n_rnn_layers" : 3,
        "rnn_dim"      : [150, 100, 50],
        "n_class"      : N_CLASSES,
        "out_ch"       : [8,8,16],
        "dropout"      : 0.3,
        "learning_rate": learning_rate,
        "batch_size"   : batch_size,
        "num_workers"  : num_workers,
        "epochs"       : epochs
        }

model = iESPnet(hparams['n_cnn_layers'],
                hparams['n_rnn_layers'],
                hparams['rnn_dim'],
                hparams['n_class'],
                hparams['out_ch'],
                hparams['dropout'],
                )

In [8]:
s=0

patients         = df_meta['rns_id'].unique().tolist()
save_path        = 'SAVEPATH/'
save_runs        = save_path + patients[s] + '/runs/'
save_models      = save_path + patients[s] + '/models/'
save_predictions = save_path + patients[s] + '/results/'
save_figs        = save_path + patients[s] + '/figs/'

In [10]:
# pregunta si esta disponible la gpu

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Using {} device'.format(device))

Using cuda device


In [18]:
 # load model

model_path = save_models + 'model_opt.pth'
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [77]:
path_results = save_predictions+ patients[s]+ 'results.npy'
path_results

results_patient_0 = np.load(path_results, allow_pickle=True)
results_patient_0=results_patient_0.item()
results_patient_0.keys()

dict_keys(['train_losses', 'train_acupr', 'prediction_te', 'prediction_tr', 'hparams', 'threshold', 'train_size'])

In [78]:
train_losses_  = results_patient_0.get("train_losses")
train_acupr_   = results_patient_0.get("train_acupr") 
prediction_te_ = results_patient_0.get("prediction_te") 
prediction_tr_ = results_patient_0.get("prediction_tr") 

In [82]:
df_train = pd.DataFrame({ 'train_losses': train_losses_, 'train_acupr':train_acupr_})
df_train

,train_losses,train_acupr
0,0.141611,0.497843
1,0.056635,0.871250
2,0.035553,0.965430
3,0.031739,0.974281
4,0.029610,0.975429
5,0.027870,0.978386
6,0.026205,0.982568
7,0.024391,0.984402
8,0.023292,0.986604
9,0.022210,0.988455


In [74]:
print(prediction_te_.keys())


dict_keys(['accuracy', 'f1', 'precision', 'recall', 'y_true', 'y_pred', 't_true', 't_pred', 'l_true', 'l_pred', 'proba', 'MAE_time'])
